<br><br><br>Jeff Smolinski<br>
10/10/2020<br>
Homework 5

# Web scrapping for Data Scientist job in CO(Total points 9)

In this exercise we'll do web scrapping for **Data Scientist job in CO**


Here is the link to the search query

https://www.indeed.com/jobs?q=data+scientist&l=CO

As you can see at the bottom of the page there are link to series of pages related to this search.
If you click on second page, search url changes to

https://www.indeed.com/jobs?q=data+scientist&l=CO<br>
https://www.indeed.com/jobs?q=data+scientist&l=CO&start=10<br>
https://www.indeed.com/jobs?q=data+scientist&l=CO&start=20<br>
https://www.indeed.com/jobs?q=data+scientist&l=CO&start=30<br>
https://www.indeed.com/jobs?q=data+scientist&l=CO&start=40<br>
https://www.indeed.com/jobs?q=data+scientist&l=CO&start=50<br>
https://www.indeed.com/jobs?q=data+scientist&l=CO&start=60<br>
https://www.indeed.com/jobs?q=data+scientist&l=CO&start=70<br>
https://www.indeed.com/jobs?q=data+scientist&l=CO&start=80<br>

If you click on 3rd then url changes to

https://www.indeed.com/jobs?q=data+scientist&l=CO&start=20

Hence to go to more pages we can format search string(**change start=??** part) for **requests.get in a loop**


In [5]:
#!curl https://www.indeed.com/robot.txt
# FILE NOT FOUND

In [1]:
#import urllib
#import urllib3
#http = urllib3.PoolManager()
#r = http.request('GET', 'http://indeed.com/robots.txt')
#r.status

import urllib.robotparser


rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.indeed.com/robot.txt')
rp.read()
rp.can_fetch('*','https://www.indeed.com/jobs?q=data+scientist&l=CO')



True

In [2]:
from bs4 import BeautifulSoup as bsoup
import requests
import pandas as pd

pages = []
url = 'https://www.indeed.com/jobs?q=data+scientist&l=CO'
url2 = 'https://www.indeed.com/jobs?q=data+scientist&l=CO&start='


response = requests.get(url)
soup = bsoup(response.text, "html.parser")
pages.append(soup.find(id='resultsCol'))
print(0)

for i in range(1,9):
    
    response = requests.get(url2+str(i*10))
    #print(response)
    soup = bsoup(response.text, "html.parser")
    #html = open(url2+str(i*10)).read()
    #soup = bsoup(html)
    pages.append(soup.find(id='resultsCol'))
    print(i)
    
    



0
1
2
3
4
5
6
7
8


# Q1 (5 =  4(non indicator columns) + 1(indicator columns) points) Please complete the following task

- Scrape 10 pages (**last page(10 th) url will be https://www.indeed.com/jobs?q=data+scientist&l=CO&start=90**)and build a pandas DataFrame containing following information
    + **job title, name of the company, location, summary of job description**
    + **Indicator columns(with value True/False) about keywords Python, SQL, AWS, RESTFUL, Machine learning, Deep Learning, Text Mining, NLP, SAS, Tableau, Sagemaker, TensorFlow, Spark**

Note:
- Make sure that you do a case insensitive search for keywords when filing(True/False) in the indicator columns
- You need to go to the webpage of detail job posting for keywords search. Main job posting only contains summary of the job description.  Build detail job posting webpage url  from web scrapping main search results.

In [186]:
df_all = pd.DataFrame()
print('RUNNING...', len(df_all),' len')

count = 0
for page_soup in pages:
    df = pd.DataFrame()
    #job_titles_obj = page_soup.find_all('a',{'class':'jobtitle'})
    job_titles = []
    company_names = []
    locations = []
    summaries = []
    descriptions = []
    #for job in page_soup.find_all('h2',{'class':'title'}):jobsearch-SerpJobCard
    
    for job in page_soup.find_all('div',{'class':'jobsearch-SerpJobCard'}):
        title = job.find('a',{'class':'jobtitle'})
        job_titles.append(title.get('title'))
        comp_name = job.find('span',{'class':'company'})
        company_names.append(comp_name.text.strip())
        loca = job.find('div',{'class':'location'})
        if loca == None:
            loca = job.find('span',{'class':'location'})
        locations.append(loca.text.strip())
        
        summary_obj = job.find('div',{'class':'summary'})
        summaries.append(summary_obj)
        
        url_des = title.get('href')
        result = url_des.find('indeed.com')
        if result == -1:
            url_des = 'https://www.indeed.com'+url_des
        #print(url_des)
        response = requests.get(url_des)
        soup = bsoup(response.text, "html.parser")
        temp = soup.find(id='jobDescriptionText')
        descriptions.append(temp.text.strip())
        
    count += 1
    print('Page: ',count)
        
        
        
    #print(job_titles)
    df['job_title'] = job_titles
    #print(company_names)
    df['company_name'] = company_names
    #print(locations)
    df['location'] = locations
    df['summary'] = summaries
    
    df['description'] = descriptions
    # ----
    '''
    #company_name_obj = page_soup.find_all('a',{'data-tn-element':'companyName'})
    company_names = []
    for name in page_soup.find_all('span',{'class':'company'}):
        company_names.append(name.text.strip())
    
    print(company_names)
    df['company_name'] = company_names
    '''
    # ----
    '''
    #location_obj = page_soup.find_all('span',{'class':'location'})
    locations = []
    for loca in page_soup.find_all('span',{'class':'location'}):
        locations.append(loca.text.strip())
    
    print(locations)
    df['location'] = locations
    '''
    # -----
    '''
    #sum_divs = page_soup.find_all()
    sum_lis = []
    for sum_div in page_soup.find_all('div',{'class':'summary'}):
        sum_lis.append(sum_div.find('li').contents[0])
    
    summary = '\n'.join(sum_lis)
    df['summary'] = summary
    '''
    #df
    #print(df)
    
    if len(df_all) == 0:
        df_all = pd.DataFrame(df)
        #print(df_all)
    else:
        #pd.merge(df_all,df, left_on='none', how='left')
        df_all = pd.concat([df_all, df], ignore_index=True)
    #df_all
    
    
#df_all.head(3)
print('done.')
#print(df_all)

RUNNING... 0  len
Page:  1
Page:  2
Page:  3
Page:  4
Page:  5
Page:  6
Page:  7
Page:  8
Page:  9
done.


In [253]:
df_all.reset_index(drop=True) # fail
df_all.reindex(axis="columns")

df_all.sample(10)


,job_title,company_name,location,summary,description
29,"Decision Scientist, CPG Analytics",Ibotta,"Denver, CO 80202 (Central Business District area)","[\n, [\n, [With over 50 people, Ibotta's Analy...",Would you like to be part of an industry-leadi...
110,Senior Data Analyst for Product,CircleCI,Colorado,"[\n, [\n, [Define requirements for , <b>data</...",As a Senior Data Analyst for Product at Circle...
111,DATA SCIENTIST MANAGER,JBS USA Food Company,"Greeley, CO","[\n, [\n, [Solid technical hands-on skills in ...",JBS USA is a leading global provider of divers...
46,Data Scientist,CACI,"Denver, CO 80225","[\n, [\n, [Apply , <b>data</b>, mining, , <b>...",Job Description\nWhat You’ll Get to Do\nBe par...
2,Data Scientist,TCS,"Broomfield, CO","[\n, [\n, [ merge , <b>data</b>, sources; en...",GRS # 162897\n\n\nRole: Data Scientist\n\n\nLo...
21,DATA SCIENTIST,Decentrix,"Denver, CO 80202 (Central Business District area)","[\n, [\n, [Collaborate with a team of software...",Decentrix offers an exciting opportunity to in...
37,Data Scientist,Maven Wave Partners,"Denver, CO","[\n, [\n, [Aptitude - You have an innate capac...","Who We Are!\nAt Maven Wave, an Atos Company, w..."
81,DATA SCIENTIST MANAGER,JBS USA Food Company,"Greeley, CO","[\n, [\n, [Solid technical hands-on skills in ...",JBS USA is a leading global provider of divers...
114,Sr. Statistician,Numerator,"Denver, CO","[\n, [\n, [Working with the survey research te...",Numerator is a data and technology company rei...
68,NLP researcher,Comcast,"Denver, CO 80202 (Lodo area)","[\n, [\n, [Apply scientific knowledge in NLP a...",Business Unit:\nNLP Researcher – Comcast Appli...


In [278]:
df_2 = df_all.copy(deep=True)

In [279]:
df_2.sample(3)

,job_title,company_name,location,summary,description
70,Senior Data Scientist,Prescient Edge Federal,"Colorado Springs, CO 80914 (Powers area)","[\n, [\n, [A competitive salary with performan...",Job Description:\n\nPrescient Edge is seeking ...
108,"GIS Data Analyst/Scientist, Task 6.9.2.4",Indrasoft,"Aurora, CO","[\n, [\n, [Knowledge of , <b>data</b>, scienc...",Job Details\nDescription\nJob Title: GIS Data ...
69,Data Architect,Berkshire Hathaway Homestate Companies,"Denver, CO 80237 (Southeastern Denver area)","[\n, [\n, [Defines governance and best practic...",WHO WE ARE:\nBerkshire Hathaway Homestate Comp...


In [256]:
keywords = 'Python, SQL, AWS, RESTFUL, Machine learning, Deep Learning, Text Mining, NLP, SAS, Tableau, Sagemaker, TensorFlow, Spark'

keys = keywords.split(', ')
#keys = [x.lower() for x in keys]
print(keys)



['Python', 'SQL', 'AWS', 'RESTFUL', 'Machine learning', 'Deep Learning', 'Text Mining', 'NLP', 'SAS', 'Tableau', 'Sagemaker', 'TensorFlow', 'Spark']


In [190]:
df_2.description

0      Req #: 20-07349\nTitle: Junior Data Scientist\...
1      Req #: 20-07348\nTitle Data Scientist III\nLoc...
2      GRS # 162897\n\n\nRole: Data Scientist\n\n\nLo...
3      Our client is a Data Analytics, a market leade...
4      The coolest jobs on this planet… or any other…...
                             ...                        
127    Embedded Machine Learning Research Engineer\nB...
128    Artificial Intelligence, Senior Consultant – A...
129    Artificial Intelligence, Consultant - Applied ...
130    BS in Computer Science or related technical fi...
131    Strategy & Analytics-Analytics and Cognitive\n...
Name: description, Length: 132, dtype: object

In [280]:

for k in keys:
    strs = df_2.description
    x = []
    #for i in strs:
        #x.append(i.str.contains(k, na=False, case=False))
    x = strs.str.contains(str(k), na=False, case=False)
    kk = k.replace(' ','_')
    print(kk)
    df_2[kk] = x
    
    

Python
SQL
AWS
RESTFUL
Machine_learning
Deep_Learning
Text_Mining
NLP
SAS
Tableau
Sagemaker
TensorFlow
Spark


In [281]:
df_2.sample(3)

,job_title,company_name,location,summary,description,Python,SQL,AWS,RESTFUL,Machine_learning,Deep_Learning,Text_Mining,NLP,SAS,Tableau,Sagemaker,TensorFlow,Spark
126,Senior Data Platform Engineer,IHS Markit,"Englewood, CO 80112","[\n, [\n, [Excited to build , <b>data</b>, pl...",Senior Data Platform Engineer\nAbout Us\nAt IH...,True,True,True,False,False,False,False,False,False,False,False,False,True
122,Senior Data Scientist,Perspecta,"Colorado Springs, CO","[\n, [\n, [Experience with machine learning, a...",Business Group Highlights\n\n\nDefense\nThe De...,True,False,False,False,True,False,False,False,False,False,False,False,False
38,Senior Data Scientist,"Operating Company ACTIVISION PUBLISHING, INC","Boulder, CO","[\n, [\n, [Advanced analytics and SQL skills.]...",The Senior Data Scientist will develop and ope...,True,True,True,False,True,True,False,False,False,False,False,False,True


In [260]:
#df_2['Machine_learning'][60:90]
df_2['Machine_learning'].value_counts()
#df_2.loc[df_2['Machine_learning'] == True]

True     84
False    48
Name: Machine_learning, dtype: int64

# Q2 (1 point) Save your DataFrame to pickle file name *indeed_job_co.pkl*. 
   Load this pkl file in dataFrame and use this dataFrame for answering following questions.

   <font color='red'>upload the pickle file(indeed_job_co.pkl) along with solution notebook to the canvas</font>

In [282]:
#write code here
import pickle
import sys
sys.setrecursionlimit(100000)
df_2.to_pickle("./indeed_job_co.pkl") #potentially_malicious.pkl

In [283]:
df_loaded2 = pd.read_pickle("./indeed_job_co.pkl")

In [284]:
df_loaded2.sample(2)

#df_2 = df_loaded2

,job_title,company_name,location,summary,description,Python,SQL,AWS,RESTFUL,Machine_learning,Deep_Learning,Text_Mining,NLP,SAS,Tableau,Sagemaker,TensorFlow,Spark
105,Machine Learning Engineer,FlexJobs,Colorado,"[\n, [\n, [3-5 Years Experience with NLP techn...","Who We Are\nIn the past 13 years, FlexJobs has...",True,True,True,False,True,True,False,True,False,False,False,True,False
117,Junior Data Scientist,Ascent Services Group,"Denver, CO 80246 (Southeastern Denver area)","[\n, [\n, [There is typically several back and...",Req #: 20-07349\nTitle: Junior Data Scientist\...,False,True,False,False,False,False,False,False,True,False,False,False,False


<font size = "6" color='red'> Use pandas functionality to answer question 2</font>
# Q3 a (1 point) Which city has maximum job posting.



<b>ANSWER:</b> Denver, final answer.

In [285]:
df_2 = df_loaded2.copy(deep=True)

In [286]:
df_2['location'].value_counts()[0:10]

Denver, CO                                           30
Boulder, CO                                          11
Denver, CO 80202 (Central Business District area)     7
Colorado                                              6
Greeley, CO                                           5
Denver, CO 80246 (Southeastern Denver area)           5
Colorado Springs, CO                                  5
Boulder, CO 80301                                     5
Englewood, CO                                         4
Aurora, CO                                            4
Name: location, dtype: int64

In [287]:
df_loaded2['location'].value_counts()[0:3]

Denver, CO                                           30
Boulder, CO                                          11
Denver, CO 80202 (Central Business District area)     7
Name: location, dtype: int64

# Q3 b (1.5 point) - Top 3 most demanding skills(like Python, AWS, SQL ...)



<b>ANSWER:</b> The top 3 skills are Machine Learning, Python, SQL.

In [289]:
df_demand = pd.DataFrame()
l1 = []
l2 = []
for i in df_loaded2.columns[5:]:
    print(i)
    #print(type(df_2[i].value_counts()))
    #dx = df_loaded2[i].value_counts()
    #dx.set_index([0], inplace=True)
    #print(dx)
    true_count = df_loaded2[i].sum()
    print(true_count)
    l1.append(i)
    l2.append(true_count)
    
    #print(dx.loc['True'])
    #print(df_2.loc[dx[i] == True].sum())
    
df_demand['skills'] = l1
df_demand['rank'] = l2
df_demand.sort_values(by=['rank'], ascending=False)[0:3]

Python
81
SQL
58
AWS
43
RESTFUL
3
Machine_learning
84
Deep_Learning
31
Text_Mining
3
NLP
16
SAS
15
Tableau
15
Sagemaker
10
TensorFlow
28
Spark
23


,skills,rank
4,Machine_learning,84
0,Python,81
1,SQL,58


# Q3 c(.5 point) What other questions you would like to ask  based on indeed data?

This is free response questions.

Can I find jobs by industry?<br>
I would add Neural Networks, Artificial Inteligence, and Robotics to the skill list.<br>
Looks like Salary information is rare... but there are occational links to salary pages for select job posting.<br><br>I could probably auto-post to all of these listings too.